# 函数装饰器和闭包
- 装饰器最好通过实现__call__方法的类实现，不应该像本章的示例那样通过函数实现

In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return func
@deco
def target():
    print('target running !')

target()

target running !


## 装饰器的一个关键特性是，它们在被装饰的函数定义之后立即运行。这通常是在导入时（即Python加载模块时）

## 局部变量 和 全局变量：

In [4]:
def f1(a):
    print(a)
    print(b)

b = 1 # b是全局变量，能够成果运行
f1(2)

2
1


In [9]:
def f2(a):
    # global b
    print(a)
    print(b)
    b = 2 #
b = 5
f2(2) # 由于 f2 对 b进行了 赋值，那么b就是 被识别成局部变量。自然不能够在没有 赋值前打印。
# 如果要进行修改，只需要对 开始的 b 进行 global 参数

2


UnboundLocalError: local variable 'b' referenced before assignment

## 闭包：
闭包指延伸了作用域的函数，其中包含函数定义体中引用、但是不在定义体中定义的非全局变量。函数是不是匿名的没有关系，关键是它能访问定义体之外定义的非全局变量

例如：

In [14]:
def make_averager():
    series = [] # 自由变量： 未在本地作用域中绑定的变量
    def averager(new_value): # 能够使用到函数体之外的函数变量
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager


In [16]:
avg = make_averager()
avg(10)
avg(20)

15.0

- 闭包是一种函数，它会保留定义函数时存在的自由变量的绑定，这样调用函数时，虽然定义作用域不可用了，但是仍能使用那些绑定。
- 只有嵌套在其他函数中的函数才可能需要处理不在全局作用域中的外部变量
- 局限性： 类似局部变量和全局变量一样， 自由变量也不能够在 嵌套函数底部被赋值（会被识别成局部变量）
- 解决办法： 使用 nonlocal 关键词：

In [23]:
def maker_averager_pro():
    count, total = 0, 0
    def averager(new_value):
        # nonlocal count, total # use nonlocal keywords
        count += 1 # local variable 'count' referenced before assignment
        total += new_value
        return total / count
    return averager
avg = maker_averager_pro()
avg(10)
avg(20)

UnboundLocalError: local variable 'count' referenced before assignment

In [24]:
import time
def clock(func): # 实现一个 装饰器：
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ','.join(repr(arg) for arg in args)
        print('[%0.8fs]%s(%s) -> %r'%(elapsed, name, arg_str, result))
        return result
    return clocked

In [25]:
@clock
def snooze(seconds):
    time.sleep(seconds)
snooze(.123)

[0.12779951s]snooze(0.123) -> None


这是装饰器的典型行为：把被装饰的函数替换成新函数，二者接受相同的参数，而且（通常）返回被装饰的函数本该返回的值，同时还会做些额外操作。
- clock装饰器有几个缺点：不支持关键字参数，而且遮盖了被装饰函数的__name__和__doc__属性
- functools.wraps装饰器把相关的属性从func复制到clocked中, 并且能够让 clocked 获取到 *关键字参数*

## 标准库中的装饰器：
- functools.lru_cache(maxsize=128, typed=False) #  做备忘录，缓存
    - 因为lru_cache使用字典存储结果，而且键根据调用时传入的定位参数和关键字参数创建，所以被lru_cache装饰的函数，它的所有参数都必须是可散列的。
- functools.singledispatch # 单分配泛函数
    - 出现原因： python 不支持 重载方法或函数
- 使用@singledispatch装饰的普通函数会变成泛函数（genericfunction）
- @singledispath的优点是支持模块化扩展：各个模块可以为它支持的各个类型注册一个专门函数。
- 各个专门函数使用@«base_function».register(«type»)装饰。

In [28]:
from functools import singledispatch
from collections import abc
import numbers
import html
@singledispatch # 注册 泛函数
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str) # 针对不同的输入，进行不同的处理： str
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p}'.format(content)
@htmlize.register(numbers.Integral) # 针对不同的输入，进行不同的处理： Integeral (注册的专门函数应该处理抽象基类)
def _(n):
    return '<pre>{0}(0x{0:x})</pre>'.format(n)


## 参数化装饰器：
- 参数化装饰器通常会把被装饰的函数替换掉，而且结构上需要多一层嵌套。

In [29]:
# 普通 装饰器：
registry = []
def register(func):
    print('running register(%s)'%func)
    registry.append(func)
    return func
@register
def f1():
    print('running f1()')
f1()

running register(<function f1 at 0x112c70048>)
running f1()


In [32]:
registry = set()
def register_pro(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)'%(active,func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate
@register_pro(active=True) # 即使不传入参数，register也必须作为函数调用（@register（　））
def f2():
    print('running f2()')
f2()

running register(active=True)->decorate(<function f2 at 0x112c9b268>)
running f2()
